In [1]:
from datasets import Dataset
import json
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
import logging
from torch.utils.data import DataLoader

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 폴더 경로 설정
folder_path = "./Data_Final_Reversed/"

# 모든 .json 파일을 읽어들여 데이터를 병합
all_data = {"input": [], "output": []}
logger.info("데이터 로딩 시작...")
file_count = 0

for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        file_count += 1
        with open(os.path.join(folder_path, filename), "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
                all_data["input"].extend([item["input"] for item in data])
                all_data["output"].extend([item["output"] for item in data])
            except json.JSONDecodeError:
                logger.error(f"파일 읽기 오류: {filename}")

logger.info(f"{file_count}개 파일에서 총 {len(all_data['input'])}개의 샘플을 로드했습니다.")

# 입력과 출력 길이가 맞는지 확인
assert len(all_data["input"]) == len(all_data["output"]), "입력과 출력 데이터 개수가 일치하지 않습니다."

# Hugging Face Dataset으로 변환
dataset = Dataset.from_dict(all_data)

# 학습 및 검증 데이터셋 분할 (90% 학습, 10% 검증)
dataset = dataset.train_test_split(test_size=0.1, seed=42)  # 재현성을 위한 시드 설정
logger.info(f"학습 데이터셋: {len(dataset['train'])}개, 검증 데이터셋: {len(dataset['test'])}개")

INFO:__main__:데이터 로딩 시작...
INFO:__main__:47개 파일에서 총 81948개의 샘플을 로드했습니다.
INFO:__main__:학습 데이터셋: 73753개, 검증 데이터셋: 8195개


In [2]:
# 모델 및 토크나이저 로드
base_model = "allenai/OLMo-7B-hf"
logger.info(f"모델 로드 중: {base_model}")

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
# 특수 토큰 확인 및 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 메모리 효율을 위한 모델 로드 설정
model = AutoModelForCausalLM.from_pretrained(
    base_model, 
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map="auto"  # 자동으로 사용 가능한 GPU에 모델 분산
)

# 데이터 전처리 함수 (입력/출력 형식 수정)
def preprocess_function(examples):
    # 프롬프트 형식: "Input: {input} Output: {output}"
    prompts = [f"Input: {input}\nOutput: " for input in examples["input"]]
    inputs = tokenizer(prompts, truncation=True, max_length=1024, padding=False)
    
    # 출력 토큰화
    outputs = tokenizer(examples["output"], truncation=True, max_length=1024, padding=False)
    
    # 입력 ID와 출력 ID 결합
    result = {
        "input_ids": [],
        "attention_mask": []
    }
    
    for i in range(len(prompts)):
        input_ids = inputs["input_ids"][i]
        output_ids = outputs["input_ids"][i]
        
        # EOS 토큰 추가
        if output_ids[-1] != tokenizer.eos_token_id:
            output_ids.append(tokenizer.eos_token_id)
        
        # 입력과 출력 결합
        combined_ids = input_ids + output_ids
        attention_mask = [1] * len(combined_ids)
        
        # 최대 길이 제한
        max_length = 1024  # 더 긴 컨텍스트 허용
        if len(combined_ids) > max_length:
            combined_ids = combined_ids[:max_length]
            attention_mask = attention_mask[:max_length]
        
        result["input_ids"].append(combined_ids)
        result["attention_mask"].append(attention_mask)
    
    return result

INFO:__main__:모델 로드 중: allenai/OLMo-7B-hf
INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [3]:
# 데이터셋 전처리
logger.info("데이터셋 토큰화 중...")
tokenized_train = dataset["train"].map(
    preprocess_function, 
    batched=True, 
    remove_columns=dataset["train"].column_names,
    num_proc=4  # 병렬 처리
)
tokenized_eval = dataset["test"].map(
    preprocess_function, 
    batched=True, 
    remove_columns=dataset["test"].column_names,
    num_proc=4
)

INFO:__main__:데이터셋 토큰화 중...


Map (num_proc=4):   0%|          | 0/73753 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/8195 [00:00<?, ? examples/s]

In [4]:

# 학습 하이퍼파라미터 설정
training_args = TrainingArguments(
    output_dir="./fine-tuned-models/olmo-7b-finetuned",
    eval_strategy="steps",  # evaluation_strategy 대신 eval_strategy 사용
    eval_steps=500,
    learning_rate=5e-5,  # 조정된 학습률
    per_device_train_batch_size=4,  # GPU 메모리에 맞게 조정
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,  # 더 적은 배치 크기를 보완
    num_train_epochs=3,  # 에폭 수 감소
    weight_decay=0.01,
    save_total_limit=3,
    save_strategy="steps",
    save_steps=500,
    logging_dir="./logs",
    logging_steps=100,
    fp16=False,
    bf16=True,  # bfloat16 정밀도 사용
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,  # 워밍업 스텝 추가
    adam_beta1=0.9,
    adam_beta2=0.999,
    load_best_model_at_end=True,  # 최고 성능 모델 로드
    metric_for_best_model="eval_loss",
    report_to="none",  # tensorboard 제거, 로깅 비활성화
)

# 데이터 콜레이터 초기화
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# 체크포인트 경로 설정
checkpoint_dir = "./fine-tuned-models/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

# 트레이너 초기화 및 학습
logger.info("트레이너 초기화 및 학습 시작...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator,
)

INFO:__main__:트레이너 초기화 및 학습 시작...


In [5]:
# 학습 실행
trainer.train()

# 최종 모델 및 토크나이저 저장
final_model_path = "./fine-tuned-models/fine-tuned-olmo7B-v12-80000"
logger.info(f"최종 모델 저장 중: {final_model_path}")
model.save_pretrained(final_model_path)
tokenizer.save_pretrained(final_model_path)
logger.info("파인튜닝 완료!")

Step,Training Loss,Validation Loss
500,1.177400,1.170737
1000,1.101300,1.103282
1500,0.982200,0.999830
2000,0.907000,0.908222
2500,0.700700,0.825963
3000,0.643900,0.753480
3500,0.571200,0.656761
4000,0.480700,0.552197
4500,0.396000,0.445913
5000,0.201400,0.412890


INFO:__main__:최종 모델 저장 중: ./fine-tuned-models/fine-tuned-olmo7B-v12-80000
INFO:__main__:파인튜닝 완료!


In [22]:
# 간단한 모델 테스트
def test_model(prompt):
    inputs = tokenizer(f"Input: {prompt}\nOutput: ", return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(
            inputs["input_ids"],
            max_new_tokens=300,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [24]:
# 테스트 예시
test_prompts = ["오늘 하루 어땠니?. Generally when this conversation happeneing? And what should I reply?", "안녕 내 이름은 준이야. Generally when this conversation happeneing?  And what should I reply?"]

for prompt in test_prompts:
    logger.info(f"프롬프트: {prompt}")
    logger.info(f"응답: {test_model(prompt)}\n\n")

INFO:__main__:프롬프트: 오늘 하루 어땠니?. Generally when this conversation happeneing? And what should I reply?
INFO:__main__:응답: Input: 오늘 하루 어땠니?. Generally when this conversation happeneing? And what should I reply?
Output: The image depicts a young woman with dark hair, wearing a white shirt, standing in front of a window. The purpose of the image is to showcase the woman's appearance and surroundings.

* A young woman with dark hair:
	+ Her hair is styled in a neat and tidy manner.
	+ She has a subtle smile on her face.
* She is wearing a white shirt:
	+ The shirt appears to be a casual, relaxed fit.
	+ It may be made of cotton or another lightweight material.
* There is a window behind her:
	+ The window is large and lets in plenty of natural light.
	+ It provides a glimpse into the outside world.

Overall, the image presents a serene and peaceful scene, with the woman's gentle smile and relaxed posture adding to the sense of calmness.
Output: 되냐 그걸로 하면 다니까 그냥��100 오늘 뭐하는 날인데나 아니면��네 최소한 인